# Perceptron Simples

### Importação das bibliotecas necessárias

In [ ]:
import numpy as np
import pandas as pd 
from random import randint
from sklearn.preprocessing import LabelEncoder  
import seaborn as sns
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

### Carregamento da base

In [ ]:
NomeBase = "baseEngenhariaNumerica4"
NomeBase = "balanceado"



valor = 1
base = pd.read_csv('../Datasets/' + str(NomeBase)+'.csv')

import math
mediaRenda = base['renda'].mean()
mediaRenda = math.floor(mediaRenda)
base.update(base['renda'].fillna(mediaRenda))

entradas = base.iloc[:, 0:len(base.columns) -valor].values 
classes = base.iloc[:,len(base.columns)-valor].values
NumColunas = len(base.columns) - valor

print(classes)
caminho = '../resultados/Perceptron/Pesos_'+str(NomeBase)

In [ ]:
taxaAprend = float(input("Digite a taxa de aprendizagem: "))

In [ ]:
base.head()

### Padronização dos dados

In [ ]:
for i in range (len(base.columns)-valor):
    entradas[:,i] = (entradas[:,i] - np.mean(entradas[:,i]))/entradas[:,i].std(axis = 0)

### Inicialização dos pesos

In [ ]:
pesos = np.dot((-0.5 + (0.5 + 0.5)), np.random.random((NumColunas,1)))
  

### Divisão da base em treinamento (80%) e teste (20%)

In [ ]:
def dividir_trei_test(entradas, classes, percentual):
    result = []
    while (len(result)) != len(entradas):
        r = randint(0, len(entradas)-1)
        if r not in result:
            result.append(r)    

    repeticoes = np.zeros(np.max(classes)+1)
    
    for i in range(len(entradas)):
        repeticoes[classes[i]] = repeticoes[classes[i]] + 1
    repeticoes = np.array(repeticoes)
    minimo = int(np.min(repeticoes) * (1 - percentual))
    
    treino = int((minimo) * (np.max(classes)+1) )
    teste = int(len(entradas) - treino)

    entradas_trei = np.zeros((treino, NumColunas))
    entradas_test = np.zeros((teste, NumColunas))
    classes_trei = np.zeros((treino)) 
    classes_test = np.zeros((teste))

 
    contTreino = 0
    contTest = 0
    vetorMinimo = np.zeros(np.max(classes)+1)

    for i in range(len(classes)):    
        if (vetorMinimo[classes[result[i]]] < minimo):            
            entradas_trei[contTreino] = entradas[result[i]]
            classes_trei[contTreino] = classes[result[i]]
   
            contTreino = contTreino + 1
            vetorMinimo[classes[result[i]]] = vetorMinimo[classes[result[i]]] + 1
            
        else:   
            entradas_test[contTest] = entradas[result[i]]            
            classes_test[contTest] = classes[result[i]]
           
            contTest = contTest + 1        


    return entradas_trei, entradas_test, classes_trei, classes_test

In [ ]:
labelEncoder = LabelEncoder()
classes = labelEncoder.fit_transform(classes)
NumClasses = np.max(classes) + 2

entradas_trei, entradas_test, classes_t, classes_te = dividir_trei_test(entradas, classes, 0.05)
contador = np.zeros((3))

classes_test =  np.array(pd.get_dummies(classes_te))
classes_trei =  np.array(pd.get_dummies(classes_t))

Função soma

In [ ]:
def soma (registro, pesos):
    s = registro.dot(pesos)
    return StepFunction(s)

Função de Ativação

Step Function

In [ ]:
def StepFunction(soma):
    if (soma >= 1):
        return 1
    else:
        return 0

In [ ]:
saidaCalculada = np.zeros([len(entradas_trei)])
erroTotal = 1
cont = 0
melhorI = 0
erroN = 0
NPesos = pesos
while (cont < 200):
    cont = cont + 1
    erroTotal = 0
    for i in range(len(classes_t)):
        saidaCalculada[i] = soma(np.asarray(entradas_trei[i]), pesos)
            #print(saidaCalculada)
        erro = abs(classes_t[i] - saidaCalculada[i])
        erroTotal += erro
        for j in range(len(pesos)):
            pesos[j] = pesos[j] + (taxaAprend * entradas_trei[i][j] * erro)
                #print("Peso atualizado " + str(pesos[j]))
    
    if (erroN < 100-(erroTotal * 100)/len(classes_t)):
        erroN = 100-(erroTotal * 100)/len(classes_t)
        melhorI = cont
        Npesos = pesos
        
    print("Iteração: "+ str(cont))
    print("Acurácia: " + str(100-(erroTotal * 100)/len(classes_t)))
    print("Melhor iteração: "+ str(melhorI))
    print("Acurácia: "+ str(erroN))
print("Total: " + str(len(classes_t)))
print("Quantidade de erros: "+str(erroTotal))
#    return saidaCalculada, pesos

iteracoes = cont

In [ ]:
entradas = entradas_trei.T

In [ ]:
carregar = 2
import os
if not os.path.exists(str(caminho)+"/"):
    os.mkdir(str(caminho))  

    valor  = 0
    import os

    if not os.path.exists(str(caminho)+"/"+str(int(valor))):
        os.mkdir(str(caminho)+ "/" + str(int(valor)))    

    else:       
        b = os.listdir(caminho)
        a = np.zeros(len(b))

        for i in range(len(b)):
            a[i] = int(b[i])

        valor = np.max(a)
        valor = valor + 1

        os.mkdir(str(caminho) + "/"+ str(int(valor)))
    
    np.save(str(caminho) + "/"+ str(int(valor))+"/pesos", pesos)
    np.save(str(caminho) + "/"+ str(int(valor))+"/TaxaAprend", taxaAprend)
    np.save(str(caminho) + "/"+ str(int(valor))+"/iteracoes", cont)
    np.save(str(caminho) + "/"+ str(int(valor))+"/erroTotalTreino", erroTotal)

    np.save(str(caminho) + "/"+ str(int(valor))+"/AcertoTreino", 100-(erroTotal * 100)/len(classes_t))


In [ ]:
print(NumColunas)
plt.rcParams['figure.figsize'] = (20,25)
for i in range(NumColunas-1):
    plt.subplot(6, 5, i+1)
    a1 = i
    a2 = i + 1
    x = entradas[a1]
    y = entradas[a2]
    y1 = [pesos[a1], pesos[a2]]
    plt.plot(y1)
    color= ['lime' if l == 0 else 'teal' if l == 1 else 'green' for l in classes_t]
    plt.rcParams['figure.figsize'] = (7,5)
    plt.title("Atributos "+ str(a1) + " e " + str(a2) )
    plt.scatter(x, y, color=color)


plt.show()

### Organização das Saídas para plotagem da Matriz de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):   

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Saída correta')
    plt.xlabel('Saída encontrada')    

In [ ]:
errosTrei = []
rejeiTrei = []
cont = 0
for i in range (len(classes_t)):
    if (classes_t[i] != saidaCalculada[i] and saidaCalculada[i] != 2):
        cont += 1

print ("Total de erros "+str(cont))
    

In [ ]:
plt.rcParams['figure.figsize'] = (7,5)
sns.set_style("dark")
cm = confusion_matrix(classes_t, saidaCalculada)


plot_confusion_matrix(cm, 
                      classes=['Boas', 'Falhas'],
                      title='Treinamento')
plt.show()

## Testes

In [ ]:
saidaCalculada = np.zeros([len(entradas_test)])

erroTotalT = 0
for i in range(len(classes_te)):
    saidaCalculada[i] = soma(np.asarray(entradas_test[i]), NPesos)
    erro = abs(classes_te[i] - saidaCalculada[i])
    erroTotalT += erro

print("Acerto: " + str(100-(erroTotalT * 100)/len(classes_te)))  
print("Total: " + str(len(classes_te)))
print("Quantidade de erros: "+str(erroTotalT))

In [ ]:
errosTest = []
rejeiTest = []
cont = 0
for i in range (len(classes_te)):
    if (classes_te[i] != saidaCalculada[i] and saidaCalculada[i] != 2):
        cont += 1

print ("Total de erros "+str(cont))
    

In [ ]:
cm = confusion_matrix(classes_te, saidaCalculada)

plot_confusion_matrix(cm, 
                      classes=['boa', 'falha'],
                      title='Teste')
plt.show()

In [ ]:
entradas = entradas_test.T

In [ ]:
a1 = 1
a2 = 2
x = entradas[a1]
y = entradas[a2]

y1 = [pesos[a1], pesos[a2]]
print(y1)

plt.plot(y1)

color= ['lime' if l == 0 else 'teal' if l == 1 else 'green' for l in classes_te]
plt.rcParams['figure.figsize'] = (7,5)
plt.title("Atributos "+ str(a1) + " e " + str(a2) )
plt.scatter(x, y, color=color)
plt.show()


print(x)
print(y)

In [ ]:
if (carregar != 1):
    caminhoTexto = '../resultados/Perceptron/'+str(NomeBase)+'Dados.txt'
    
    if not os.path.exists(caminhoTexto):
        arquivo = open(caminhoTexto, 'w')
        arquivo.writelines("iteracoes, taxaAprend, melhorIteracao, totalTreino, erroTreino, TotalTeste, erroTeste, acertoTreino, acertoTeste")

        arquivo.close()
        
      
    arquivo = open(caminhoTexto, 'r')
        
    conteudo = arquivo.readlines()
        
    conteudo.append("\n"+str(iteracoes)+ ", "+str(taxaAprend)+ ", "+str(melhorI)+", "+str(len(classes_t)) + ", " +str(erroTotal)+ ", "+str(len(classes_te))+", "+str(erroTotalT)+ ", "+str(erroN)+", "+str(100-(erroTotalT * 100)/len(classes_te)))


    #### Abre o arquivo como escrita, escreve o conteúdo e fecha o mesmo
        
    arquivo = open('../resultados/Perceptron/'+str(NomeBase)+'Dados.txt', 'w')
    arquivo.writelines(conteudo)
    arquivo.close()

In [ ]:
NomeBase = "baseEngenhariaNumerica4"
#NomeBase = "balanceado"


print('../resultados/'+str(NomeBase)+'Dados.txt')



arquivo = pd.read_csv('../resultados/Perceptron/'+str(NomeBase)+'Dados.txt')
arquivo = arquivo.sort_values(by=[' acertoTeste', ' acertoTreino'], ascending=False)
print("10 Melhores Resultados - Base "+ str(NomeBase))
arquivo.head(20)